In [1]:
import requests
from bs4 import BeautifulSoup
import time as time
import urllib.request
import random
from getpass import getpass
import os
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
SEARCH_TAGS = ["에어컨추천", "에어프라이기추천", "인테리어가전"]
SCROLL_NUM = 5

In [4]:
from Insta_Crawl import *

In [5]:
insta = Insta_Crawl(driver_loc = "../webdriver/chromedriver_99.exe")

로그인 완료


In [6]:
temp = insta.collect_insta(tags = SEARCH_TAGS, img_dir = None)

Tag : 에어컨추천, # of URL : 57
0,1,2,3,Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
에어컨추천_goldlady_rh08_2022-04-28_img_1.jpg
4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,HTTPSConnectionPool(host='scontent-gmp1-1.cdninstagram.com', port=443): Max retries exceeded with url: /v/t51.2885-15/279472852_449595653636087_1791673258713711071_n.webp?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=nAiG4bJjnSsAX8cCD3H&edm=AABBvjUBAAAA&ccb=7-4&ig_cache_key=MjgyNjcwODA1MjI1MjM0Njg0OA%3D%3D.2-ccb7-4&oh=00_AT9Rk1KC-4noD48euMhy6RVBsX1CfbMmSBMe1PbSQhX6ug&oe=6273357B&_nc_sid=83d603 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001EFF8938B20>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))
에어컨추천_goldlady_rh08_2022-04-28_img_3.jpg
52,53,54,55,56,
Tag : 에어프라이기추천, # of

In [18]:
insta.driver.find_elements(By.XPATH, "//div[@class='KL4Bh']/img")

<selenium.webdriver.remote.webelement.WebElement (session="2fe809a99ff491ffaa08caf6ab974cda", element="c8f1c406-7484-4cfb-be37-da8723159a31")>

In [10]:
insta.driver.find_elements(By.XPATH, "//button[@class='  _6CZji    ']")[0].click()

In [7]:
class Insta_Crawl:
    SCROLL_NUM = 5
    
    def __init__(self, driver):
        self.driver = driver
        self.__USERNAME = input("Input Instagram ID : ")
        self.__PASSWORD = getpass("Input Password : ")
        
        self.login()


    def login(self):
        self.driver.get("https://www.instagram.com/accounts/login/")

        if self.driver.current_url == "https://www.instagram.com/":
            print("You already loged in")
            return None
        
        username_form = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, "//input[@name='username']")))
        username_form.send_keys(self.__USERNAME)

        password_form = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
        password_form.send_keys(self.__PASSWORD)

        login_button = self.driver.find_element(By.XPATH, "//button[@class='sqdOP  L3NKy   y3zKF     ']")
        login_button.click()

        driver.implicitly_wait(5)

        # 로그인 정보 저장 팝업 스킵
        if self.driver.current_url == "https://www.instagram.com/accounts/onetap/?next=%2F":
            skip_save_button = self.driver.find_element(By.XPATH, "//button[@class='sqdOP yWX7d    y3zKF     ' or contains(text(), '나중에 하기')]")
            skip_save_button.click()

        # # 알림 설정 팝업 스킵
        # try:
        #     skip_alarm_button = driver.find_element(By.XPATH, "//button[@class='aOOlW   HoLwm ' or contains(text(), '나중에 하기')]")
        #     skip_alarm_button.click()
        # except NoSuchElementException:
        #     pass

        search_bar = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='eyXLr wUAXj ']")))
        

    
    def search_tag(self, tag):
        # 검색
        self.driver.get(f"https://www.instagram.com/tags/{tag}/")
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div/h1[@class='_7UhW9       fKFbl yUEEX   KV-D4          uL8Hv         ']")))

        # 스크롤
        for _ in range(SCROLL_NUM):
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(1)

        # 게시물 post들의 url 수집
        post_elems = self.driver.find_elements(By.XPATH, "//*[contains(@href, '/p/')]")
        post_urls = [x.get_attribute("href") for x in post_elems]

        return list(set(post_urls))
    


    # 이미지 url
    def collect_img_urls(self):
        self.img_urls = []
        while True:
            img_elems = self.driver.find_elements(By.XPATH, "//div[@class='KL4Bh']/img[contains(@srcset, '1080w') and contains(@alt, 'Photo')]")
            self.img_urls.extend([x.get_attribute("src") for x in img_elems])

            # 한 포스트에 사진이 여러개 있는 경우 오른쪽 버튼을 눌러 사진 수집
            # slide_right_button = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//button[@class='  _6CZji   ']")))
            slide_right_button = self.driver.find_elements(By.XPATH, "//button[@class='  _6CZji   ']") # 존재하지 않는 Element를 찾을 때 시간이 오래걸림..
            if len(slide_right_button) != 0:
                slide_right_button[0].click()
            else:
                break
        
        self.img_urls = list(set(self.img_urls))

        return self.img_urls



    def collect_text(self):
        text_dict = defaultdict(list)
        # 작성자 이름
        writer_name = self.driver.find_element(By.XPATH, "//a[@class='sqdOP yWX7d     _8A5w5   ZIAjV ']").text

        # 작성글
        try:
            post_body = self.driver.find_element(By.XPATH, "//div[@class='C4VMK']/span").text
        except:
            post_body = None

        # 좋아요
        try:
            like = self.driver.find_element(By.XPATH, "//div[@class='Nm9Fw']/a/span").text
        except:
            like = None

        # 날짜
        post_date = self.driver.find_element(By.XPATH, "//a[@class='c-Yi7']").text

        text_dict["writer_name"] = writer_name
        text_dict["post_body"] = post_body
        text_dict["like"] = like
        text_dict["post_date"] = post_date

        return text_dict



    def save_images(self, img_urls, img_names, img_dir = "./image/"):
        
        if img_dir is None:
            img_dir = f"./image/"
        
        if not os.path.exists(img_dir):
            os.makedirs(img_dir)

        self.error_list = []
        for img_url, img_name in zip(img_urls, img_names):
            # image 저장
            try:
                img_rq = requests.get(img_url, verify = False)
                with open(f"{img_dir}/{img_name}", "wb") as f:
                    f.write(img_rq.content)
            except: 
                self.error_list.append(img_name)
                print(img_name)



    def collect_insta(self, tags, img_dir = None):
        if not isinstance(tags, list):
            self.tags = [tags]
        else:
            self.tags = tags
        
        
        self.text_dict = defaultdict(list)
        # Tag가 여러개 들어오면 1개씩 Loop로 수집
        for tag in self.tags:

            img_dir = os.path.join("./image", tag)
            post_urls = self.search_tag(tag)
            print(f"Tag : {tag}, # of URL : {len(post_urls)}")
            # 각 Post마다 방문해서 이미지 링크 수집
            for i, url in enumerate(post_urls):
                print(i, end = ",")
                driver.get(url)

                # 이미지 url 수집
                img_urls = self.collect_img_urls()
                # 이미지가 없으면 해당 게시물은 skip (동영상인 경우도 스킵)
                if len(img_urls) == 0:
                    continue
                
                # post 내의 텍스트 수집
                self.texts_in_post = self.collect_text()

                self.text_dict["tag"].append(tag)
                self.text_dict["writer_name"].append(self.texts_in_post["writer_name"])
                self.text_dict["post_body"].append(self.texts_in_post["post_body"])
                self.text_dict["like"].append(self.texts_in_post["like"])
                self.text_dict["post_date"].append(self.texts_in_post["post_date"])
                self.text_dict["url"].append(url)

                img_names = [f"{tag}_{self.texts_in_post['writer_name']}_{self.texts_in_post['post_date'].replace(' ', '_')}_img_{str(n+1)}.jpg" for n in range(len(img_urls))]
                self.text_dict["images"].append(img_names)

                self.save_images(img_urls, img_names, img_dir)

            print("", end = "\n")
            # Excel로 저장                    
            pd.DataFrame(self.text_dict).to_excel(f"{tag}.xlsx", index = False, encoding = "CP949")

        return self.text_dict

In [65]:
temp1 = defaultdict(list)
temp2 = defaultdict(list)

temp1["a"] = [1,2]
temp2["a"] = [3,4]
temp2["b"] = [3,4]

In [61]:
temp1.update(temp2)

In [66]:
temp1

defaultdict(list, {'a': [1, 2]})

In [15]:
driver.find_elements(By.XPATH, "//video[@class='tWeCl']")[0].get_attribute("src")

'https://scontent-gmp1-1.cdninstagram.com/v/t50.16885-16/244129589_442671800641091_7455007615594894565_n.mp4?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=aNFoEksQzD4AX8GdtPI&edm=AABBvjUBAAAA&ccb=7-4&oe=61825442&oh=093ccff1f8ad5a46fe2368462c390093&_nc_sid=83d603'

In [8]:
# Login
driver.get("https://www.instagram.com/accounts/login/")

username_form = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@name='username']")))
username_form.send_keys(USERNAME)

password_form = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
password_form.send_keys(PASSWORD)

login_button = driver.find_element(By.XPATH, "//button[@class='sqdOP  L3NKy   y3zKF     ']")
login_button.click()

driver.implicitly_wait(5)

# 로그인 정보 저장 팝업 스킵
if driver.current_url == "https://www.instagram.com/accounts/onetap/?next=%2F":
    skip_save_button = driver.find_element(By.XPATH, "//button[@class='sqdOP yWX7d    y3zKF     ' or contains(text(), '나중에 하기')]")
    skip_save_button.click()


# 알림 설정 팝업 스킵
try:
    skip_alarm_button = driver.find_element(By.XPATH, "//button[@class='aOOlW   HoLwm ' or contains(text(), '나중에 하기')]")
    skip_alarm_button.click()
except NoSuchElementException:
    pass

search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='eyXLr wUAXj ']")))

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x005D9943+2595139]
	Ordinal0 [0x0056C9F1+2148849]
	Ordinal0 [0x00464528+1066280]
	Ordinal0 [0x00490FD4+1249236]
	Ordinal0 [0x004911CB+1249739]
	Ordinal0 [0x004BD812+1431570]
	Ordinal0 [0x004ABA34+1358388]
	Ordinal0 [0x004BBAF2+1424114]
	Ordinal0 [0x004AB806+1357830]
	Ordinal0 [0x00486086+1204358]
	Ordinal0 [0x00486F96+1208214]
	GetHandleVerifier [0x0077B232+1658114]
	GetHandleVerifier [0x0083312C+2411516]
	GetHandleVerifier [0x0066F261+560433]
	GetHandleVerifier [0x0066E366+556598]
	Ordinal0 [0x0057286B+2173035]
	Ordinal0 [0x005775F8+2192888]
	Ordinal0 [0x005776E5+2193125]
	Ordinal0 [0x005811FC+2232828]
	BaseThreadInitThunk [0x75C36739+25]
	RtlGetFullPathName_UEx [0x77998E7F+1215]
	RtlGetFullPathName_UEx [0x77998E4D+1165]


In [ ]:
def search_tag(tag):
    # 검색
    driver.get(f"https://www.instagram.com/tags/{tag}/")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div/h1[@class='_7UhW9       fKFbl yUEEX   KV-D4          uL8Hv         ']")))

    # 스크롤
    for _ in range(SCROLL_NUM):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)

    # 게시물 post들의 url 수집
    post_elems = driver.find_elements(By.XPATH, "//*[contains(@href, '/p/')]")
    post_urls = [x.get_attribute("href") for x in post_elems]

    return list(set(post_urls))

In [11]:
# 이미지 url
def collect_img_urls():
    img_urls = []
    while True:
        img_elems = driver.find_elements(By.XPATH, "//div[@class='KL4Bh']/img[contains(@srcset, '1080w') and contains(@alt, 'Photo')]")
        img_urls.extend([x.get_attribute("src") for x in img_elems])

        # slide_right_button = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//button[@class='  _6CZji   ']")))
        slide_right_button = driver.find_elements(By.XPATH, "//button[@class='  _6CZji   ']")
        if len(slide_right_button) != 0:
            slide_right_button[0].click()
        else:
            break

    return list(set(img_urls))


In [22]:
def collect_text():
    # 작성자 이름
    writer_name = driver.find_element(By.XPATH, "//a[@class='sqdOP yWX7d     _8A5w5   ZIAjV ']").text

    # 작성글
    try:
        post_body = driver.find_element(By.XPATH, "//div[@class='C4VMK']/span").text
    except:
        post_body = None

    # 좋아요
    try:
        like = driver.find_element(By.XPATH, "//div[@class='Nm9Fw']/a/span").text
    except:
        like = None

    # 날짜
    post_date = driver.find_element(By.XPATH, "//a[@class='c-Yi7']").text

    return {"writer_name" : writer_name, "post_body" : post_body, "like" : like, "post_date" : post_date}

In [45]:
def collect_insta(tag, loc = None):
    post_urls = search_tag(tag)
    
    # 각 Post마다 방문해서 이미지 링크 수집
    text_dict = defaultdict(list)
    for url in post_urls:
        driver.get(url)

        # 이미지 url
        img_urls = collect_img_urls()
        # 이미지가 없으면 해당 게시물은 skip (동영상인 경우도 스킵)
        if len(img_urls) == 0:
            continue
        
        texts = collect_text()
        text_dict["tag"].append(tag)
        text_dict["writer_name"].append(texts["writer_name"])
        text_dict["post_body"].append(texts["post_body"])
        text_dict["like"].append(texts["like"])
        text_dict["post_date"].append(texts["post_date"])
        text_dict["url"].append(url)

        img_names = [f"{tag}_{texts['writer_name']}_{texts['post_date'].replace(' ', '_')}_img_{str(n+1)}.jpg" for n in range(len(img_urls))]
        text_dict["images"].append(img_names)
        # print(texts['writer_name'])


        loc = f"./image/{tag}"
        if not os.path.exists(loc):
            os.makedirs(loc)
        error_list = []
        # for n, img_url in enumerate(img_urls):
        for img_url, img_name in zip(img_urls, img_names):
            # image 저장
            try:
                with urllib.request.urlopen(img_url) as f:
                    with open(f"{loc}/{img_name}", "wb") as h:
                        img = f.read()
                        h.write(img)
            except: 
                error_list.append(img_name)
                print(img_name)
                
    pd.DataFrame(text_dict).to_excel(f"{tag}.xlsx", index = False, encoding = "CP949")

    return text_dict

In [47]:
for tag in SEARCH_TAGS:
    print(tag)
    text_dict = collect_insta(tag)

LG홈브루


In [67]:
text_dict

defaultdict(list,
            {'tag': ['LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루',
              'LG홈브루'],
             'writer_name': ['hyun9982',
              'fromkmin',
          

In [222]:
pd.DataFrame(text_dict)

,writer_name,post_body,like,post_date,images
0,hyewon___cho,.\n방을 확 뒤어퍼써\n요리따윈 안한지 오래라 스매그 방으로 이동 이럴줄알았음 작...,128,2017년 5월 31일,"[스매그냉장고_hyewon___cho_2017년_5월_31일_img_1.jpg, 스..."
1,jenny_kkkk,.\n.\n.\n#홈메이드 #스매그냉장고 #주방놀이 #14개월아기 #예쁜딸 #둘째는...,18,2019년 4월 22일,[스매그냉장고_jenny_kkkk_2019년_4월_22일_img_1.jpg]
2,mona_thestudio,#smegfridge #콜라보레이션 #스매그냉장고 #리빙디자인페어2019 .\n.\...,5,2019년 4월 5일,"[스매그냉장고_mona_thestudio_2019년_4월_5일_img_1.jpg, ..."
3,jplanet.atelier,봄이오네요 오긴오네요 많이 기다렸는데\n\n#세상상회도오늘부터공사시작하고\n#곧~있...,121,2018년 3월 10일,[스매그냉장고_jplanet.atelier_2018년_3월_10일_img_1.jpg]
4,_kyoung_im_,스매그냉장고 🤓\n\n드뎌\n사고 친거 다 알고\...,67,2020년 10월 4일,[스매그냉장고__kyoung_im__2020년_10월_4일_img_1.jpg]
5,_kyoung_im_,우리집 스매그 냉장고!!! 크림색 이쁘다.\n등을 내렸더니 또 좋다.\n아 집에서 ...,19,2020년 2월 18일,[스매그냉장고__kyoung_im__2020년_2월_18일_img_1.jpg]
6,skys5699,ㆍ\nㆍ\n\n또다시. 사고싶어지는\n스매그 냉장고.\n\n이쁘니까\n다\n용서된다...,78,2018년 3월 28일,[스매그냉장고_skys5699_2018년_3월_28일_img_1.jpg]
7,roggiek,#봄봄\n계절을 느끼는 우리동네😍😍\n#스매그냉장고 몇년에 한번씩 고장나는 덕분에 ...,25,4월 30일,"[스매그냉장고_roggiek_4월_30일_img_1.jpg, 스매그냉장고_roggi..."
8,skys5699,ᆞ\nᆞ\n\n#스매그냉장고\n#이쁘다,64,2017년 9월 23일,[스매그냉장고_skys5699_2017년_9월_23일_img_1.jpg]
9,roggiek,파스텔 그린 컬러 대신 가져다 놓은 새빨간 대체냉장고 🚫🚫 게다가 작은 사이즈로 가...,21,2018년 7월 25일,[스매그냉장고_roggiek_2018년_7월_25일_img_1.jpg]
